In [ ]:
# ==========================================
# 1. Imports & Config
# ==========================================

from pathlib import Path
import json
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Paths
NB_DIR = Path.cwd()
DATA_DIR = (NB_DIR / "../data").resolve()
RES_DIR  = (NB_DIR / "../results").resolve()
RES_DIR.mkdir(parents=True, exist_ok=True)

CLEAN_CSV = DATA_DIR / "clean_chess_games.csv"           # preferred
SUBSET_CSV = DATA_DIR / "raw_chess_games_subset.csv"     # fallback

print("DATA_DIR:", DATA_DIR)
print("RESULTS:", RES_DIR)

# Sklearn / plotting / models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
import joblib

# Keras for DL model (works locally & on Colab)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.random.set_seed(RANDOM_STATE)